In [72]:
from pyscal.core import System
import json
import yaml
import numpy as np

In [82]:
def _serialise_arrays(data):
    """
    Iteratively serialise a given array
    
    Parameters
    ----------
    data : list, ndarray
        input list
        
    Returns
    -------
    data
    """
    if isinstance(data, np.ndarray):
        data = list(data)
    if isinstance(data, list):
        for c in range(len(data)):
            data[c] = _serialise_arrays(data[c])
    return data

def get_angle(vec1, vec2):
    """
    Get angle between two vectors in degrees
    
    Parameters
    ----------
    vec1: list
        first vector
    
    vec2: list
        second vector
    
    Returns
    -------
    angle: float
        angle in degrees
    
    Notes
    -----
    Angle is rounded to two decimal points
    
    """
    return np.round(np.arccos(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))*180/np.pi, decimals=2)

def write_file(outfile, data):
    """
    Write a given dict as json file
    
    Parameters
    ----------
    outfile: string
        name of output file. `.json` will be added to the given file name
    
    data: dict
        input data dict
    
    Returns
    -------
    None
    """
    with open(".".join([outfile, "json"]), "w") as fout:
        json.dump(convert_to_dict(sys), fout)
    #with open(".".join([outfile, "yaml"]), "w") as fout:
    #    yaml.unsafe_dump(convert_to_dict(sys), fout)
        
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    #not available
    info["latticeparameter"] = None
    
    #needs to be defined
    info["Occupancy"] = _serialise_arrays(sys.atoms.species)
    info["Element"] = _serialise_arrays(np.unique(sys.atoms.species))
    info["CellVolume"] = float(sys.volume)
    info["NumberOfAtoms"] = int(sys.natoms)
    
    #available 
    info["FirstAxisComponent"] = _serialise_arrays(sys.box[0])
    info["SecondAxisComponent"] = _serialise_arrays(sys.box[1])
    info["ThirdAxisComponent"] = _serialise_arrays(sys.box[2])
    info["LatticeParameterLengthA"] = _serialise_arrays(sys.boxdims[0])
    info["LatticeParameterLengthB"] = _serialise_arrays(sys.boxdims[1])
    info["LatticeParameterLengthC"] = _serialise_arrays(sys.boxdims[2])
    
    info["LatticeParameterAngleAlpha"] = _serialise_arrays(get_angle(sys.box[0], sys.box[1]))
    info["LatticeParameterAngleBeta"] = _serialise_arrays(get_angle(sys.box[1], sys.box[2]))
    info["LatticeParameterAngleGamma"] = _serialise_arrays(get_angle(sys.box[2], sys.box[0]))
    
    info["X_AxisCoordinate"] = _serialise_arrays(np.array(sys.atoms.positions)[:,0])
    info["Y_AxisCoordinate"] = _serialise_arrays(np.array(sys.atoms.positions)[:,1])
    info["Z_AxisCoordinate"] = _serialise_arrays(np.array(sys.atoms.positions)[:,2])
    
    return info

In [48]:
sys = System()
sys.read_inputfile("al_data/Al.poscar", format="poscar")

In [83]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))